In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
import os
import json
import numpy as np

In [3]:
import spacy
import spacy_transformers
nlp = spacy.load("en_core_sci_md")

In [4]:
# # Spacy Tokenize the sentences of reference articles:
# for file in os.listdir("Articles"):
#     if file.endswith('txt'):
#         print(file)
#         with open(f"Articles/{file}") as f:
#             text = f.read()
#             text = text[text.find('\n')+2:]
#             doc = nlp(text)
#             sentences_all = [str(s) for s in list(doc.sents) if str(s) != '\n\n']
#             sentences = []
#             for s in sentences_all:
#                 if "references" in s.lower():
#                     break
#                 sentences.append(s)
#             sentences = [s.strip() for s in sentences if s.strip().count(' ') >= 2]
            
#         with open(f'Sentences/{file[:-4]}.json', 'w') as g:
#             json.dump(sentences, g)




In [5]:
pwd

'/Users/shrut/Work/NewData/IR'

In [6]:
with open("sent_context.json") as f:
    con = json.load(f)

In [5]:
# # Spacy Tokenize the sentences of reference articles:
# for file in os.listdir("Articles"):
#     if file.startswith('0') and file.endswith('txt'):
#         print(file)
#         with open(f"Articles/{file}") as f:
#             text = f.read()
#             text = text[text.find('\n')+2:]
#             doc = nlp(text)
#             sentences_all = [str(s) for s in list(doc.sents) if str(s) != '\n\n']
#             sentences = []
#             for s in sentences_all:
#                 if "references" in s.lower():
#                     break
#                 sentences.append(s)
#             sentences = [s.strip() for s in sentences if s.strip().count(' ') >= 2]
        
#         output_name = file[4:][:-4]
#         with open(f'Sentences/{output_name}.json', 'w') as g:
#             json.dump(sentences, g)

031_PMC7536538.txt
036_PMC7225046.txt
024_PMC7588823.txt
029_PMC7906955.txt
026_PMC7462354.txt
040_PMC7144857.txt
041_PMC7194244.txt
022_PMC7328313.txt
028_PMC7488216.txt
032_PMC7675243.txt
038_PMC7124956.txt
023_PMC7604159.txt
030_PMC7165103.txt
025_PMC7173821.txt
034_PMC8136807.txt
035_PMC6954302.txt
027_PMC7775777.txt
039_PMC7239666.txt
037_PMC7808278.txt
033_PMC7267384.txt


In [14]:
# for file in os.listdir("Articles"):
#     if file.startswith("0"):
#         new_name = file.split("_")[1]
#         os.rename(f"Articles/{file}", f"Articles/{new_name}")

In [19]:
results = {}
def get_relevant_sentences(cs, num_tfidf=10):
    global results

    for pmcid in cs:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        else:
            curr_ref_file = pmcid
            
        CURR_PMC = pmcid
        

        with open(f'Sentences/{curr_ref_file}.json') as f:
            sentences = json.load(f)

        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(sentences)

        results[CURR_PMC] = {}
        idx = 0
        for file, d in cs[pmcid].items():
            if len(d['support']) == 0:
                continue
            claim = con[pmcid][file]
            idx += 1
            
            y = vectorizer.transform([claim])
            cosine_similarities = linear_kernel(y, X).flatten()
            related_docs_indices = cosine_similarities.argsort()[::-1]
            current_sentences = []
            for i in range(0, num_tfidf):
                current_sentences.append(sentences[related_docs_indices[i]])
            results[CURR_PMC][file] = current_sentences


In [14]:
def calculate_recall(cs, results):
    def overlap(start, end, s, e):
        return start <= e and s <= end

    for pmcid in results:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        else:
            curr_ref_file = pmcid
        with open(f'Articles/{curr_ref_file}.txt') as f:
            content = f.read()
        for file in results[pmcid]:
            indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in results[pmcid][file]]
            count = 0
            support_indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in cs[pmcid][file]['support']]
            for start, end in support_indices:
                for s, e in indices:
                    if overlap(start, end, s, e):
                        count += 1
            cs[pmcid][file]['count'] = count
    res = 0
    total = 0
    for pmcid in cs:
        for file in cs[pmcid]:
            if 'count' in cs[pmcid][file]:
                res += (cs[pmcid][file]['count'] / len(cs[pmcid][file]['support']))
                total += 1

    return res/total

In [15]:
with open('recon_cs.json') as f:
    cs = json.load(f)

In [16]:
with open('recon_cs_no_title.json') as f:
    cst = json.load(f)

In [18]:
with open('Eugene_ContextAndSupport.json') as f:
    cs1 = json.load(f)

In [19]:
with open('Eugene_ContextAndSupport_No_Title.json') as f:
    cs2 = json.load(f)

In [20]:
with open('Degaulle_ContextAndSupport.json') as f:
    cs3 = json.load(f)

In [21]:
with open('Degaulle_ContextAndSupport_No_Title.json') as f:
    cs4 = json.load(f)

In [25]:
results = {}
get_relevant_sentences(cs1)
calculate_recall(cs1, results)

0.2552227342549923

In [26]:
results = {}
get_relevant_sentences(cs2)
calculate_recall(cs2, results)

0.41884057971014493

In [42]:
results = {}
get_relevant_sentences(cs1, 5)
calculate_recall(cs1, results)

0.17680491551459293

In [43]:
results = {}
get_relevant_sentences(cs2, 5)
calculate_recall(cs2, results)

0.2900966183574879

In [27]:
results = {}
get_relevant_sentences(cs3)
calculate_recall(cs3, results)

0.41779661016949154

In [28]:
results = {}
get_relevant_sentences(cs4)
calculate_recall(cs4, results)

0.4401785714285715

In [17]:
results = {}
get_relevant_sentences(cs)
calculate_recall(cs, results)

0.43079399141630903

In [18]:
results = {}
get_relevant_sentences(cst)
calculate_recall(cst, results)

0.5436241610738256

In [20]:
results = {}
get_relevant_sentences(cs)
calculate_recall(cs, results)

0.4141630901287553

In [21]:
results = {}
get_relevant_sentences(cst)
calculate_recall(cst, results)

0.5221476510067116

In [22]:
pwd

'/Users/shrut/Work/NewData/IR'